# Optimisation 1. Reducing the noise by dropping the outliers

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Look at APPLICATION_TYPE value counts for droping unwanted values
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Reducing the noise eliminating outliers
application_types_to_drop = list(app_counts[app_counts<500].index) 

application_types_to_drop




['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [5]:
# dropping outliers in DataFrame for APPLICATION_TYPE

df2 = pd.DataFrame({'d': application_types_to_drop})

for a in application_df.APPLICATION_TYPE.index:
    for search in df2.d:
        if search in application_df.loc[a].APPLICATION_TYPE:
            application_df.drop(a, inplace=True)
            break  # so if we already dropped it we no longer look whether we should drop this line


In [6]:
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
Name: APPLICATION_TYPE, dtype: int64

In [7]:
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17081
C2000     6045
C1200     4836
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
classifications_to_drop = list(class_counts[class_counts<1000].index)

In [9]:
# dropping outliers in DataFrame for CLASSIFICATION
df3 = pd.DataFrame({'d': classifications_to_drop})

for a in application_df.CLASSIFICATION.index:
    for search in df3.d:
        if search in application_df.loc[a].CLASSIFICATION:
            application_df.drop(a, inplace=True)
            break  # so if we already dropped it we no longer look whether we should drop this line



In [10]:
application_df['CLASSIFICATION'].value_counts()

C1000    17081
C2000     6045
C1200     4836
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Finding data types in the DataFrame
types_list = application_df.dtypes
types_list

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [12]:
for item in types_list.index:
    if types_list[item] == 'int64':
        item
        types_list = types_list.drop(item)
types_list

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
dtype: object

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=types_list.index)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
input_dimensions = X_train_scaled.shape[1]
input_dimensions

41

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=input_dimensions))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 336       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 489
Trainable params: 489
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
745/745 [==============================] - 2s 1ms/step - loss: 0.6164 - accuracy: 0.6940
Epoch 2/100
745/745 [==============================] - 1s 1ms/step - loss: 0.5758 - accuracy: 0.7201
Epoch 3/100
745/745 [==============================] - 1s 995us/step - loss: 0.5690 - accuracy: 0.7212
Epoch 4/100
745/745 [==============================] - 1s 1ms/step - loss: 0.5666 - accuracy: 0.7239
Epoch 5/100
745/745 [==============================] - 1s 976us/step - loss: 0.5644 - accuracy: 0.7221
Epoch 6/100
745/745 [==============================] - 1s 955us/step - loss: 0.5636 - accuracy: 0.7228
Epoch 7/100
745/745 [==============================] - 1s 961us/step - loss: 0.5628 - accuracy: 0.7237
Epoch 8/100
745/745 [==============================] - 1s 950us/step - loss: 0.5620 - accuracy: 0.7248
Epoch 9/100
745/745 [==============================] - 1s 973us/step - loss: 0.5614 - accuracy: 0.7229
Epoch 10/100
745/745 [==============================] - 1s 969us/step - loss: 0

745/745 [==============================] - 1s 949us/step - loss: 0.5512 - accuracy: 0.7289
Epoch 81/100
745/745 [==============================] - 1s 939us/step - loss: 0.5512 - accuracy: 0.7302
Epoch 82/100
745/745 [==============================] - 1s 930us/step - loss: 0.5509 - accuracy: 0.7282
Epoch 83/100
745/745 [==============================] - 1s 986us/step - loss: 0.5512 - accuracy: 0.7294
Epoch 84/100
745/745 [==============================] - 1s 937us/step - loss: 0.5506 - accuracy: 0.7281
Epoch 85/100
745/745 [==============================] - 1s 929us/step - loss: 0.5507 - accuracy: 0.7299
Epoch 86/100
745/745 [==============================] - 1s 937us/step - loss: 0.5506 - accuracy: 0.7301
Epoch 87/100
745/745 [==============================] - 1s 926us/step - loss: 0.5505 - accuracy: 0.7300
Epoch 88/100
745/745 [==============================] - 1s 973us/step - loss: 0.5507 - accuracy: 0.7313
Epoch 89/100
745/745 [==============================] - 1s 965us/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

249/249 - 0s - loss: 0.5688 - accuracy: 0.7157 - 421ms/epoch - 2ms/step
Loss: 0.5688287019729614, Accuracy: 0.7156529426574707


## Optimisation 1 has reduced the accuracy from 72.5% to 71.5%. Therefore dropping the outliers is not recommended and a different strategy is needed

# Optimisation 2: increasing the number of layers

In [23]:
application_df2 = pd.read_csv("../Resources/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df2.drop(['EIN', 'NAME'], axis=1)
application_df2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [25]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df2['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [26]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(app_counts[app_counts<500].index) 

# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [27]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df2['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [28]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(class_counts[class_counts<1000].index) 

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [29]:
# Finding data types in the DataFrame
types_list = application_df2.dtypes
types_list

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [30]:
for item in types_list.index:
    if types_list[item] == 'int64':
        item
        types_list = types_list.drop(item)
types_list

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
dtype: object

In [31]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df2 = pd.get_dummies(application_df2, columns=types_list.index)
application_df2

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
# Split our preprocessed data into our features and target arrays
X = application_df2.drop('IS_SUCCESSFUL', axis=1).values
y = application_df2['IS_SUCCESSFUL'].values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)

In [33]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
input_dimensions = X_train_scaled.shape[1]
input_dimensions

43

# Compile, Train and Evaluate the Model

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=input_dimensions))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 8)                 352       
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
 dense_6 (Dense)             (None, 8)                 72        
                                                                 
 dense_7 (Dense)             (None, 8)                 72        
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6070 - accuracy: 0.6761
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5635 - accuracy: 0.7269
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5579 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 997us/step - loss: 0.5518 - accuracy: 0.7321
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 1000us/step - loss: 0.5503 - accuracy: 0.7324
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - a

Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7353
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.7362
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7360
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7357
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5435 - accuracy: 0.7357
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7367
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7351
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7364
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7362
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5433

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7265 - 372ms/epoch - 1ms/step
Loss: 0.5541239380836487, Accuracy: 0.7265306115150452


## Optimisation 2 has increased the accuracy from 72.5% to 72.6%, but I have still not achieved the 75% accuracy target

# Optimisation 3: Auto-tuning hyperparameters.


In [46]:
#Letting keras-tuner decide activation function, number of layers and number of neurons

In [39]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=80,
        step=5), activation=activation, input_dim=input_dimensions))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [40]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    overwrite=True,
    max_epochs=20,
    hyperband_iterations=2,)


In [41]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 35s]
val_accuracy: 0.7255976796150208

Best val_accuracy So Far: 0.7269970774650574
Total elapsed time: 00h 16m 24s
INFO:tensorflow:Oracle triggered exit


In [42]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 46,
 'num_layers': 2,
 'units_0': 86,
 'units_1': 96,
 'units_2': 41,
 'units_3': 66,
 'units_4': 11,
 'units_5': 66,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [43]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5537 - accuracy: 0.7270 - 495ms/epoch - 2ms/step
Loss: 0.5536656975746155, Accuracy: 0.7269970774650574


In [45]:
# Export our model to HDF5 file
best_model.save('../HDF5_files/AlphabetSoupCharity_Optimization.h5')

## The accuracy is 72.69%, still not 75%